Downloaded a bunch images off of ESA image archive denoting either a star or galaxy, will attempt to train a CNN to successfully determine each. 

In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import cv2
import matplotlib.pyplot as plt

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def resize_with_pad(img, img_size):
    height, width, _ = img.shape
    ratio = img_size / max(height, width)
   # print(ratio)
    if ratio < 1:
        img = cv2.resize(img, (int(ratio * width), int(ratio * height)))
    padding = ((img_size - img.shape[0], 0), (img_size - img.shape[1],0), (0,0))

    return np.pad(img, padding, 'constant')


In [35]:

galaxy_data_dir = '/Users/noahkasmanoff/Desktop/galaxies/'
star_data_dir = '/Users/noahkasmanoff/Desktop/allrise/'
# = 'data/validation'

In [36]:
X_galaxies = []
y_galaxies = []
for file in os.listdir(galaxy_data_dir):
    if file.endswith('.jpg'):
        img = cv2.imread(galaxy_data_dir + file)
        print(file)
   # print(np.shape(img))

        img = resize_with_pad(img,360)
    
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
       # img = normalize(img,norm='l1',axis=1)
        X_galaxies.append((img))

        y_galaxies.append([0,1])
X_stars = []
y_stars = []
for file in os.listdir(star_data_dir):
    if file.endswith('.jpg'):
        img = cv2.imread(star_data_dir + file)
  #  print(np.shape(img))
        img = resize_with_pad(img,360)
   # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_stars.append(img)
        y_stars.append([1,0])

heic0206a.jpg
heic0206b.jpg
heic0406a.jpg
heic0506a.jpg
heic0602a.jpg
heic0604a.jpg
heic0604d.jpg
heic0706a.jpg
heic0711a.jpg
heic0719a.jpg
heic0804a.jpg
heic0810ae.jpg
heic0817a.jpg
heic0901a.jpg
heic0905a.jpg
heic0910i.jpg
heic0910s.jpg
heic0911b.jpg
heic0911c.jpg
heic0912a.jpg
heic0914a.jpg
heic1006a.jpg
heic1104a.jpg
heic1107a.jpg
heic1110a.jpg
heic1202a.jpg
heic1215b.jpg
heic1218a.jpg
heic1302a.jpg
heic1305a.jpg
heic1403a.jpg
heic1414a.jpg
heic1421a.jpg
heic1502a.jpg
heic1503a.jpg
heic1516a.jpg
heic1709a.jpg
heic1712a.jpg
heic1801a.jpg
heic1810a.jpg
heic1811a.jpg
opo0207a.jpg
opo0415a.jpg
opo1724a.jpg
opo9925a.jpg
potw1029a.jpg
potw1035a.jpg
potw1036a.jpg
potw1238a.jpg
potw1246a.jpg
potw1329a.jpg
potw1332a.jpg
potw1348a.jpg
potw1402a.jpg
potw1408a.jpg
potw1419a.jpg
potw1423a.jpg
potw1424a.jpg
potw1425a.jpg
potw1427a.jpg
potw1435a.jpg
potw1437a.jpg
potw1438a.jpg
potw1502a.jpg
potw1536a.jpg
potw1538a.jpg
potw1726a.jpg
potw1747a.jpg
potw1811a.jpg
potw1812a.jpg


# Stars at [1,0], Galaxies are [0,1]

In [37]:
X = np.concatenate((X_galaxies,X_stars))
y = np.concatenate((y_galaxies,y_stars))


In [38]:
X = np.array((X)) 
X = X.reshape((len(X),360*360,3))
X = X.reshape(X.shape[0],360,360,3).astype('float32')


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,shuffle=True)


#Some data normalization. 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


Now to construct a CNN. I don't know if this architecture is correct, but based on the other examples I've studied it looks like my main issue is the lack of data. The runtime of a task like this is enormous, so I've been doing it on google Colab with the free GPU. 

In [45]:
model = Sequential()
model.add(Convolution2D(10, 36, 36, input_shape=(360,360,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (36, 36), input_shape=(360, 360,...)`
  from ipykernel import kernelapp as app
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [46]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [47]:
model.fit(X_train,y_train,batch_size=20,epochs=5,validation_split=.2)

Train on 101 samples, validate on 26 samples
Epoch 1/5
 80/101 [======================>.......] - ETA: 1:12 - loss: 8.3904 - acc: 0.4500

KeyboardInterrupt: 

And assuming this works, I will combine this with the images generated in the other notebook in order to identify the type of object in the LSST image. 